In [1]:
# | hide
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from pylaps.read import read_uu, read_grid, read_EBM, read_parallel_info
from pylaps.plot import plot_2d_map
from pylaps.functions import current_density, calc_deriv, VpVg_fs
from tqdm.auto import tqdm
import os
from pathlib import Path

In [4]:
# dir = "p_th_bomb"
dir = "bipolar_pulse"
gamma = 5./3.
p0 = 0.2
rho0 = 1.0

ca = 1
cs = np.sqrt(gamma * p0 / rho0)

In [5]:
file = Path(__vsc_ipynb_file__)  # type: ignore # noqa: F821
os.chdir(file.parent / dir)
os.makedirs("figures", exist_ok=True)

In [6]:
xgrid, ygrid = read_grid()
nx, ny = len(xgrid), len(ygrid)
print("nx, ny =", nx, ny)

XX, YY = np.meshgrid(xgrid, ygrid, indexing='ij')

kx = np.fft.rfftfreq(nx, xgrid[1] - xgrid[0])
ky = np.fft.rfftfreq(ny, ygrid[1] - ygrid[0])

npe, nvar = read_parallel_info()
print("npe, nvar = ", npe, nvar)

t_EBM, radius, Ur = read_EBM()

nx, ny = 256 256
npe, nvar =  4 8


In [7]:
def over_plot(ax, t, x0, y0, cs=1, ca=1):
    result = VpVg_fs(cs, ca)
    Cg_slow_perp = result["Vgs_perp"]
    Cg_slow_para = result["Vgs_para"]
    Cg_fast_perp = result["Vgf_perp"]
    Cg_fast_para = result["Vgf_para"]

    # plot group velocity of fast/slow wind
    ax.scatter(
        Cg_slow_para * t + x0,
        Cg_slow_perp * t + y0,
        s=0.5,
        color="orange",
        label=r"$V_{g,s}$",
    )
    ax.scatter(
        Cg_fast_para * t + x0,
        Cg_fast_perp * t + y0,
        s=0.5,
        label=r"$V_{g,f}$",
    )

In [8]:
def save(name):
    fname = Path(f"./figures/{name}.png")
    fname.parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(fname, dpi=300)
    plt.close()


def plot_2d_maps(
    file,
    plot_rho=True,
    plot_p=True,
    plot_Babs=True,
    plot_Bz=True,
    plot_Jx=True,
    plot_Jx_Alfven=True,
):
    t, uu = read_uu(file, nx, ny, nvar)
    rho, ux, uy, uz, Bx, By, Bz, p = [uu[:, :, i] for i in range(8)]

    Babs = np.sqrt(Bx**2 + By**2 + Bz**2)
    rho1, p1, Babs1 = [arr - np.mean(arr) for arr in (rho, p, Babs)]

    x0 = np.mean(XX)
    y0 = np.mean(YY)

    round_t = round(t, 2)

    # Plot 2D color-coded maps
    if plot_rho:
        fig, ax = plot_2d_map(XX, YY, rho1, r"$\delta \rho$" + f" $t={t:.2f}$")
        over_plot(ax, t, x0, y0, cs=cs)
        save(f"rho1_t_{t:.2f}")

    if plot_p:
        fig, ax = plot_2d_map(XX, YY, p1, r"$\delta P$" + f" $t={t:.2f}$")
        over_plot(ax, t, x0, y0, cs=cs)
        save(f"p1_t_{t:.2f}")

    if plot_Babs:
        fig, ax = plot_2d_map(XX, YY, Babs1, r"$\delta |B|$" + f" $t={t:.2f}$")
        over_plot(ax, t, x0, y0, cs=cs)
        save(f"Babs1_t_{t:.2f}")

    if plot_Bz:
        fig, ax = plot_2d_map(XX, YY, Bz, r"$B_z$" + f" $t={t:.2f}$")
        over_plot(ax, t, x0, y0, cs=cs)
        save(f"Bz_t_{t:.2f}")

    if plot_Jx:
        Jx, Jy, Jz = current_density(Bx, By, Bz, xgrid, ygrid)
        fig, ax = plot_2d_map(XX, YY, Jx, r"$J_x$" + f" $t={t:.2f}$")
        save(f"Jx/t_{round_t}")

    if plot_Jx_Alfven:
        Ey = -(uz * Bx - ux * Bz)
        dEy_dy = calc_deriv(Ey, ygrid, dim=1)
        Jx_Alfven = dEy_dy / ca
        fig, ax = plot_2d_map(XX, YY, Jx_Alfven, r"$J_{x,Alfven}$" + f" $t={t:.2f}$")
        save(f"Jx_Alfven/t_{round_t}")

In [9]:
files = sorted(glob.glob("./diags/out*dat"))
nout = len(files)

# Loop through files and process data
for file in tqdm(files):
    # plot_2d_maps(file)
    plot_2d_maps(file, plot_rho=False, plot_p=False, plot_Babs=False, plot_Bz=False, plot_Jx=True)

100%|██████████| 31/31 [00:29<00:00,  1.03it/s]
